In [41]:
#!/usr/bin/env python3
import pandas as pd

# load data
fp_data = '/home/amait/Téléchargements/Test_technique_data_engineer_Affluences/data.csv'
data = pd.read_csv(fp_data, delimiter=',')
fp_time = '/home/amait/Téléchargements/Test_technique_data_engineer_Affluences/timetables.csv'
timetables = pd.read_csv(fp_time, delimiter=',')

df_data = data.copy()
df_time = timetables.copy()
# applay to_datetime
today = pd.to_datetime("today")
df_data.last_record_datetime = df_data.last_record_datetime.apply(pd.to_datetime)

df_data.last_record_datetime = data.last_record_datetime.apply(pd.to_datetime)
df_data['last_record_datetime2'] = today - df_data.last_record_datetime

df_data['alert'] = today - df_data.last_record_datetime
mask0 = df_data.last_record_datetime2 < pd.to_timedelta("0 days 02:00:00.00000")
mask1 = (df_data.last_record_datetime2 >= pd.to_timedelta("0 days 02:00:00.00000")) & (df_data.last_record_datetime2 < pd.to_timedelta("1 days 00:00:00.00000"))
mask2 = (df_data.last_record_datetime2 >= pd.to_timedelta("1 days 00:00:00.00000")) & (df_data.last_record_datetime2 < pd.to_timedelta("2 days 00:00:00.00000"))
mask3 = df_data.last_record_datetime2 > pd.to_timedelta("2 days 00:00:00.00000")
df_data['alert'][mask0] = 0
df_data['alert'][mask1] = 1
df_data['alert'][mask2] = 2
df_data['alert'][mask3] = 3

'''
n = df_data.shape[0]
for i in range(n)
    if test_horaire(df_data.iloc[i, :]):
        pass
        print(blabla)
'''
df_data.head(10)

<ipython-input-41-9167466c4c3c>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['alert'][mask0] = 0
<ipython-input-41-9167466c4c3c>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['alert'][mask1] = 1
<ipython-input-41-9167466c4c3c>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data['alert'][mask2] = 2


,sensor_identifier,sensor_name,site_id,last_record_datetime,last_record_datetime2,alert
0,3,sensor_3,388,2019-01-30 13:55:00,826 days 04:14:10.295168,3.0
1,4,sensor_4,389,2019-01-30 14:00:00,826 days 04:09:10.295168,3.0
2,7,sensor_7,1784,2021-03-25 06:11:00,41 days 11:58:10.295168,3.0
3,8,sensor_8,1784,2021-05-04 15:17:00,1 days 02:52:10.295168,2.0
4,9,sensor_9,443,2021-05-04 15:15:00,1 days 02:54:10.295168,2.0
5,10,sensor_10,446,2021-05-04 15:15:00,1 days 02:54:10.295168,2.0
6,11,sensor_11,440,2021-05-04 15:15:00,1 days 02:54:10.295168,2.0
7,11,sensor_11,1279,2021-05-04 15:15:00,1 days 02:54:10.295168,2.0
8,13,sensor_13,441,2021-05-04 15:15:00,1 days 02:54:10.295168,2.0
9,13,sensor_13,1279,2021-05-04 15:15:00,1 days 02:54:10.295168,2.0
